In [0]:
from google.colab import drive
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Transform xml data into pandas dataframe

In [0]:
import xml.sax
import xml.etree.ElementTree as ET
import pandas

class GroupHandler(xml.sax.ContentHandler):
    def __init__(self):
      self.data = {'Id':[],
              'Score':[],
              'FavoriteCount':[],
              'ViewCount':[],
              'AnswerCount':[],
              'CommentCount':[]}

      self.inTags = False
      self.currentTag = ""

    def startElement(self, name, attrs):
        if name == "Tags":
            self.inTags = True
        if not self.inTags:
          self.currentTag = name
          

    def characters(self, content):
      if not self.inTags:
        if self.currentTag == "Id":
            self.data["Id"].append(content)        
        if self.currentTag == "Score":
            self.data["Score"].append(content)
        if self.currentTag == "FavoriteCount":
            self.data["FavoriteCount"].append(content)
        if self.currentTag == "ViewCount":
            self.data["ViewCount"].append(content)
        if self.currentTag == "AnswerCount":
            self.data["AnswerCount"].append(content)
        if self.currentTag == "CommentCount":
            self.data["CommentCount"].append(content)

    def endElement(self,name):
        if self.inTags and name == "Tags":
          self.inTags = False


handler = GroupHandler()
parser = xml.sax.make_parser()
parser.setContentHandler(handler)
parser.parse('drive/My Drive/onlab/SmallData_noText_works.xml')

print()

print(list(map( lambda key: (key,len(handler.data[key])),handler.data.keys())))

# Create DataFrame
dataFrame = pandas.DataFrame(handler.data)
 
# Print the output.
print(dataFrame)



[('Id', 100), ('Score', 100), ('FavoriteCount', 100), ('ViewCount', 100), ('AnswerCount', 100), ('CommentCount', 100)]
     Id Score FavoriteCount ViewCount AnswerCount CommentCount
0     4   645            49     45103          13            3
1     6   290            11     18713           6            0
2     9  1754           438    574705          61            5
3    11  1461           537    152779          37            3
4    13   597           147    182042          24           10
..  ...   ...           ...       ...         ...          ...
95  761    17             0       693           7            0
96  766    35             3      3095           6            0
97  769    38             5     33044          10            1
98  773   463           204    264508          12            1
99  805    14             1      2073           5            0

[100 rows x 6 columns]


Normalize data

In [0]:
from sklearn.preprocessing import MinMaxScaler

normalizedDataFrame = dataFrame
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(dataFrame) 
normalizedDataFrame.loc[:,:] = scaled_values
print(normalizedDataFrame)

          Id     Score  FavoriteCount  ViewCount  AnswerCount  CommentCount
0   0.000000  0.119124       0.007535   0.075588     0.200000      0.176471
1   0.002497  0.052105       0.001692   0.030938     0.083333      0.000000
2   0.006242  0.328488       0.067354   0.971655     1.000000      0.294118
3   0.008739  0.273173       0.082577   0.257772     0.600000      0.176471
4   0.011236  0.110062       0.022605   0.307284     0.383333      0.588235
..       ...       ...            ...        ...          ...           ...
95  0.945069  0.000566       0.000000   0.000448     0.100000      0.000000
96  0.951311  0.003965       0.000461   0.004512     0.083333      0.000000
97  0.955056  0.004531       0.000769   0.055185     0.150000      0.058824
98  0.960050  0.084765       0.031370   0.446813     0.183333      0.058824
99  1.000000  0.000000       0.000154   0.002783     0.066667      0.000000

[100 rows x 6 columns]


In [0]:
# from sklearn.preprocessing import StandardScaler

# standardisedDataFrame = dataFrame
# scaler = StandardScaler() 
# scaled_values = scaler.fit_transform(dataFrame) 
# standardisedDataFrame.loc[:,:] = scaled_values
# print(standardisedDataFrame)

Create training and validation sets

In [0]:
dataset=dataFrame.values
test_split=0.1
valid_split=0.1

inputData = dataset[:,1:6]

validation_index = int(inputData.shape[0]*(1-valid_split-test_split))
test_index = int(inputData.shape[0]*(1-test_split))

inputData_test = inputData[test_index:]
inputData_valid = inputData[validation_index:test_index]
inputData = inputData[:validation_index]

print(inputData)
#print(inputData_valid)
#print(inputData_test)


[[1.19124032e-01 7.53498385e-03 7.55883796e-02 2.00000000e-01
  1.76470588e-01]
 [5.21049651e-02 1.69152699e-03 3.09375159e-02 8.33333333e-02
  0.00000000e+00]
 [3.28487823e-01 6.73535291e-02 9.71654569e-01 1.00000000e+00
  2.94117647e-01]
 [2.73173494e-01 8.25772720e-02 2.57772025e-01 6.00000000e-01
  1.76470588e-01]
 [1.10062299e-01 2.26049516e-02 3.07283894e-01 3.83333333e-01
  5.88235294e-01]
 [7.38153672e-02 8.76518530e-03 2.13938379e-01 1.50000000e-01
  2.35294118e-01]
 [2.22767604e-02 2.15285253e-03 1.39229819e-01 1.00000000e-01
  0.00000000e+00]
 [3.15272796e-02 3.07550361e-03 1.23672233e-01 1.33333333e-01
  1.76470588e-01]
 [5.68246177e-02 1.23020145e-02 8.19704584e-02 3.66666667e-01
  9.41176471e-01]
 [2.88842741e-02 3.84437952e-03 1.24907365e-01 8.33333333e-02
  0.00000000e+00]
 [2.49197659e-02 1.07642626e-03 1.97688781e-02 1.33333333e-01
  0.00000000e+00]
 [1.32150274e-02 3.07550361e-04 1.53850058e-02 1.16666667e-01
  0.00000000e+00]
 [2.32206910e-02 3.84437952e-03 9.726240

In [0]:
# XML pars valuable data
#         -> Word2Vec
#         -> Tag implementation

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# Early Stopping to avoid overfitting the training data
patience=40
early_stopping=EarlyStopping(patience=patience, verbose=1)
# Save the weights
checkpointer=ModelCheckpoint(filepath="weights.hdf5", save_best_only=True, verbose =1)

# Define Model
model = Sequential()

# Define Loss-function

Using TensorFlow backend.


In [0]:
# Train the model

# Validate the model

In [0]:
# After training is finished
# Get the encoding model (discard the decoder model)

# Do hieracrchical clustering

# Visualize results